In [38]:
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from scipy import optimize
from scipy import special

In [3]:
def normal_dist(x , mean , var):
    prob_density = 1 / (np.sqrt(2*np.pi)*var) * np.exp(-0.5*((x-mean)/var)**2)
    return prob_density

def wasserstein(mu1, var1, mu2, var2):
    return np.sqrt(
        ( mu1 - mu2 )**2 + ( np.sqrt(var1) - np.sqrt(var2) )**2
    )

# Standard GV-fuzzy metric

\begin{align*}
    M_d(x, y, t) &= \frac{ t }{ t + d(x,y) }
\end{align*}

In [21]:
def standard_gv_fuzzy(t ,mu1, var1, mu2, var2):
    return np.array([ u / ( u + wasserstein(mu1, var1, mu2, var2) ) if u > 0 else u for u in t ])

In [25]:
@widgets.interact(mu1=(-10,10,0.1),var1=(1e-9,2,0.1),mu2=(-10,10,0.1),var2=(1e-9,2,0.1))
def show_standard_gv(mu1=0, var1=1, mu2=0.5, var2=1.5):
    x = np.linspace(-5, 5)

    pdf1 = normal_dist(x, mu1, var1)
    pdf2 = normal_dist(x, mu2, var2)

    t = np.linspace(0, 50)
    difference = standard_gv_fuzzy(t, mu1, var1, mu2, var2)

    plt.figure(figsize=(24,9))
    plt.subplot(3,1,1)
    plt.plot(x, pdf1, label='PDF 1')
    plt.plot(x, pdf2, label='PDF 2')
    plt.grid()
    plt.legend()

    plt.subplot(3,1,2)
    plt.plot(t, difference)
    plt.ylim(0, 1.1)
    plt.grid()

    plt.subplot(3,1,3)
    plt.plot(t[:-1], np.diff(difference))
    plt.ylim(0, 1.1)
    plt.grid()
    plt.show()

interactive(children=(FloatSlider(value=0.0, description='mu1', max=10.0, min=-10.0), FloatSlider(value=1.0, d…

## Fitting the exponential distribution

In [31]:
def exp_pdf(x, l):
    g = l * np.exp( - l * x )
    return g

def exp_cdf(x, l):
    g = np.array([1 - np.exp( - l * u ) if u >= 0 else 0 for u in x ])
    return g

In [36]:
@widgets.interact(mu1=(-10,10,0.1),var1=(1e-9,2,0.1),mu2=(-10,10,0.1),var2=(1e-9,2,0.1))
def show_standard_gv(mu1=0, var1=1, mu2=0.5, var2=1.5):
    x = np.linspace(-5, 5)

    pdf1 = normal_dist(x, mu1, var1)
    pdf2 = normal_dist(x, mu2, var2)

    t = np.linspace(0, 50)
    difference = standard_gv_fuzzy(t, mu1, var1, mu2, var2)

    popt, pcov = optimize.curve_fit(exp_cdf, t, difference)

    plt.figure(figsize=(24,9))
    plt.subplot(3,1,1)
    plt.plot(x, pdf1, label='PDF 1')
    plt.plot(x, pdf2, label='PDF 2')
    plt.grid()
    plt.legend()

    plt.subplot(3,1,2)
    plt.plot(t, difference, label='difference')
    plt.plot(t, exp_cdf(t, *popt), label=r'$F(x; \lambda)$')
    plt.ylim(0, 1.1)
    plt.legend()
    plt.grid()

    plt.subplot(3,1,3)
    plt.plot(t[:-1], np.diff(difference), label='Difference')
    plt.plot(t[:-1], exp_pdf(t[:-1], *popt), label='exp distr.')
    plt.ylim(0, 1.1)
    plt.legend()
    plt.grid()
    plt.show()

interactive(children=(FloatSlider(value=0.0, description='mu1', max=10.0, min=-10.0), FloatSlider(value=1.0, d…

## Fitting the Gamma distribution

In [52]:
def gamma_cdf(t, k, theta):
    return special.gammainc(k, t / theta) / special.gamma(k)

def gamma_pdf(x, k, theta):
    g = np.array([ ( u**(k-1) * np.exp( -u / theta ) ) / ( theta**k * special.gamma(k) ) if u > 0 else 1.0 for u in x])
    return g

In [53]:
@widgets.interact(mu1=(-10,10,0.1),var1=(1e-9,2,0.1),mu2=(-10,10,0.1),var2=(1e-9,2,0.1))
def show_standard_gv(mu1=0, var1=1, mu2=0.5, var2=1.5):
    x = np.linspace(-5, 5)

    pdf1 = normal_dist(x, mu1, var1)
    pdf2 = normal_dist(x, mu2, var2)

    t = np.linspace(0, 50)
    difference = standard_gv_fuzzy(t, mu1, var1, mu2, var2)

    popt, pcov = optimize.curve_fit(gamma_cdf, t, difference)

    plt.figure(figsize=(24,9))
    plt.subplot(3,1,1)
    plt.plot(x, pdf1, label='PDF 1')
    plt.plot(x, pdf2, label='PDF 2')
    plt.grid()
    plt.legend()

    plt.subplot(3,1,2)
    plt.plot(t, difference, label='difference')
    plt.plot(t, gamma_cdf(t, *popt), label=r'$F(x; k, \theta)$')
    plt.ylim(0, 1.1)
    plt.legend()
    plt.grid()

    plt.subplot(3,1,3)
    plt.plot(t[:-1], np.diff(difference), label='Difference')
    plt.plot(t[:-1], gamma_pdf(t[:-1], *popt), label=r'$f(x; k, \theta)$')
    plt.ylim(0, 1.1)
    plt.legend()
    plt.grid()
    plt.show()

interactive(children=(FloatSlider(value=0.0, description='mu1', max=10.0, min=-10.0), FloatSlider(value=1.0, d…

# Exponential GV-fuzzy metric

\begin{align*}
    M_e(x, y, t) &= e^{ - \frac{ d(x,y) }{ t } }
\end{align*}

In [26]:
def exponential_gv_fuzzy(t, mu1, var1, mu2, var2):
    return np.array([ np.exp( - wasserstein(mu1, var1, mu2, var2) / u ) if u > 0 else 0 for u in t])

In [28]:
@widgets.interact(mu1=(-10,10,0.1),var1=(1e-9,2,0.1),mu2=(-10,10,0.1),var2=(1e-9,2,0.1))
def show_standard_gv(mu1=0, var1=1, mu2=0.5, var2=1.5):
    x = np.linspace(-5, 5, 100)

    pdf1 = normal_dist(x, mu1, var1)
    pdf2 = normal_dist(x, mu2, var2)

    t = np.linspace(0, 50, 100)
    difference = exponential_gv_fuzzy(t, mu1, var1, mu2, var2)

    plt.figure(figsize=(24,9))
    plt.subplot(3,1,1)
    plt.plot(x, pdf1, label='PDF 1')
    plt.plot(x, pdf2, label='PDF 2')
    plt.grid()
    plt.legend()

    plt.subplot(3,1,2)
    plt.plot(t, difference)
    plt.ylim(0, 1.1)
    plt.grid()

    plt.subplot(3,1,3)
    plt.plot(t[:-1], np.diff(difference))
    plt.ylim(0, 1.1)
    plt.grid()
    plt.show()

interactive(children=(FloatSlider(value=0.0, description='mu1', max=10.0, min=-10.0), FloatSlider(value=1.0, d…

## Fitting the eponential distribution

In [37]:
@widgets.interact(mu1=(-10,10,0.1),var1=(1e-9,2,0.1),mu2=(-10,10,0.1),var2=(1e-9,2,0.1))
def show_standard_gv(mu1=0, var1=1, mu2=0.5, var2=1.5):
    x = np.linspace(-5, 5)

    pdf1 = normal_dist(x, mu1, var1)
    pdf2 = normal_dist(x, mu2, var2)

    t = np.linspace(0, 50)
    difference = exponential_gv_fuzzy(t, mu1, var1, mu2, var2)

    popt, pcov = optimize.curve_fit(exp_cdf, t, difference)

    plt.figure(figsize=(24,9))
    plt.subplot(3,1,1)
    plt.plot(x, pdf1, label='PDF 1')
    plt.plot(x, pdf2, label='PDF 2')
    plt.grid()
    plt.legend()

    plt.subplot(3,1,2)
    plt.plot(t, difference, label='difference')
    plt.plot(t, exp_cdf(t, *popt), label=r'$F(x; \lambda)$')
    plt.ylim(0, 1.1)
    plt.legend()
    plt.grid()

    plt.subplot(3,1,3)
    plt.plot(t[:-1], np.diff(difference), label='Difference')
    plt.plot(t[:-1], exp_pdf(t[:-1], *popt), label='exp distr.')
    plt.ylim(0, 1.1)
    plt.legend()
    plt.grid()
    plt.show()

interactive(children=(FloatSlider(value=0.0, description='mu1', max=10.0, min=-10.0), FloatSlider(value=1.0, d…